## Merge Cluster Numbers with Original Data


In [1]:
import pandas as pd

In [9]:
#we anticipate having multiple cluster lists for different cutoff 
CLUST_PATHS = ["/shared/3/projects/newsDiffusion/data/interim/NEREmbedding/embeddingClusterList2020.pkl"]
CLUST_SPECS = ["2_1000_80"]
DF_PATH = "/shared/3/projects/newsDiffusion/data/processed/newsData/fullDataWithNERCleaned.tsv"
OUT_PATH = "/shared/3/projects/newsDiffusion/data/processed/fullDataWithClustNumsTest.tsv"

In [10]:
#load in main data source 
#we don't want to use "content", because it takes up a lot of space and
#we have already embedded the content. Can always merge back in later so long as we 
#keep the "key" column
LOAD_COLS = list(pd.read_csv(DF_PATH, \
                     nrows = 1, sep="\t").columns)
#LOAD_COLS.remove("content")
#LOAD_COLS.remove("NamedEntities")

df = pd.read_csv(DF_PATH,\
                 sep="\t", usecols=LOAD_COLS)

<ipython-input-10-6c45e2367119>:10: DtypeWarning: Columns (3,4,11,12,14,15,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DF_PATH,\


In [15]:
df = df.drop(columns=["NamedEntities"])

In [20]:
#keep merging in our new columns to the dataframe 
for i, clustPath in enumerate(CLUST_PATHS): 
    currDf = pd.read_pickle(clustPath)
    currDf.rename(columns={"clustNum":CLUST_SPECS[i]})
    df = pd.merge(df, currDf, on="key", how="left")

In [21]:
df["date"] = pd.to_datetime(df["date"])

df["year"] = df["date"].dt.year


In [8]:
df = df.drop(columns=["Unnamed: 0"])
df.to_csv(OUT_PATH, sep="\t")